# Lab 1.2: Pipeline from the Database to ArcGIS Online

## Importing Packages

In [1]:
import arcpy
import psycopg2
from psycopg2 import sql

## Connecting to the Server

In [2]:
connection = psycopg2.connect(host = '34.132.44.118',
                              database = 'lab1-2',
                              user = 'postgres',
                              password = 'password')

In [3]:
# testing server connection - 0 means it is connected
connection.closed

0

## Connecting to the Cursor

In [4]:
cursor = connection.cursor()

## Creating Points

In [5]:
NW_Harriet = arcpy.Point(-93.31593708, 44.92888419)
NE_Harriet = arcpy.Point(-93.29519335, 44.92877064)
SE_Harriet = arcpy.Point(-93.29433794, 44.91480186)
SW_Harriet = arcpy.Point(-93.31689941, 44.91616482)

In [6]:
l = [NW_Harriet,NE_Harriet,SE_Harriet,SW_Harriet]
l

[<Point (-93.31593708, 44.92888419, #, #)>, <Point (-93.29519335, 44.92877064, #, #)>, <Point (-93.29433794, 44.91480186, #, #)>, <Point (-93.31689941, 44.91616482, #, #)>]

In [7]:
a = arcpy.Array(l)

In [8]:
poly = arcpy.Polygon(a)

## Creating the Table & Importing Data

In [9]:
cursor.execute("DROP TABLE IF EXISTS harriet_poly;")

In [10]:
new_table = sql.SQL("CREATE TABLE harriet_poly (polygon_id serial, polygon geometry)")
cursor.execute(new_table)

In [16]:
cursor.execute("INSERT INTO harriet_poly(polygon) VALUES (ST_GeomFromText('{}'))".format(poly.WKT))

In [17]:
cursor.execute("SELECT ST_SetSRID(polygon, 4326) FROM harriet_poly;")

In [18]:
# Committing and closing the cursor

connection.commit()
cursor.close()
connection.close()

## Converting the WKT to a GeoJSON

In [111]:
# I have commented out this tep because I am doing this in my .py instead
#cursor = connection.cursor()
#cursor.execute("SELECT ST_AsGeoJSON(harriet_poly.*) FROM harriet_poly;")
#connection.commit()

## Create the Flask App

In [ ]:
'''
# DO NOT RUN IN ARCPRO
# THIS IS NOT UP-TO-DATE WITH .PY FILE

import string
from flask import Flask, jsonify, request
import psycopg2

app = Flask(__name__)

connection = psycopg2.connect(host = '34.132.44.118',
                              database = 'lab1-2',
                              user = 'postgres',
                              password = 'password')

@app.route('/asgeojson', methods=['GET', 'POST'])

def get_geojson():
    cursor = connection.cursor()
    cursor.execute("SELECT ST_AsGeoJSON(harriet_poly.*) AS poly_geojson FROM harriet_poly;")
    result = cursor.fetchone()
    return result
    
    if result is None:
        return jsonify({'error': 'Polygon not found'}), 404
    else:
        return jsonify({'geojson': result[0]})
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
'''

## Exporting the GeoJSON from the Website

In [34]:
# This step has been fixed since my video - we resolved it during class

import requests
geojson_url = r'http://35.184.33.216:5000/asgeojson'
cwd = r'C:\Users\tande\Documents\GitHub\GIS5572\Lab01'
rqst_objct = requests.get(geojson_url)


with open (cwd + '\harriet_poly.geojson', 'w') as file:
    file.write(str(req_text.json()))
    print('GeoJSON saved!')

ConnectionError: HTTPConnectionPool(host='35.184.33.216', port=5000): Max retries exceeded with url: /asgeojson (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000025F78FF37C0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

## Incomplete Steps

## Fixing the Extracted GeoJSON

In [ ]:
# This step has been fixed since my video - we resolved it during class

# The GeoJSON is exporting as <bound method Response.json of <Response [200]>> so I need to find a way to extract the true GeoJSON

In [ ]:
# Try 1

pathtogeojson = r'http://35.184.33.216:5000/asgeojson'
post_req_obj = requests.post(pathtogeojson)

In [33]:
# Try 2
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

session.get(pathtogeojson)

# Ref https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests

ConnectionError: HTTPConnectionPool(host='35.184.33.216', port=5000): Max retries exceeded with url: /asgeojson (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000025F78DB8490>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

## Convert the GeoJSON to an ESRI Compatible Format

In [1]:
# This step has been fixed since my video - we resolved it during class

# I never got to this step becasue I was unable to correctly export my GeoJSON

arcpy.conversion.JSONToFeatures('harriet_poly.geojson', 'harriet_poly')

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input JSON or GeoJSON: Dataset harriet_poly.geojson does not exist or is not supported
Failed to execute (JSONToFeatures).


## Importing the GeoJSON Into ArcGIS Online

In [36]:
# I tried doing this by just copying and pasting the URL into ArcGIS Online and got an error
# Which is when I realized I was not getting the correct json format